In [1]:
# Helper libraries
from __future__ import absolute_import, division, print_function
import numpy as np
import matplotlib.pyplot as plt
import os
from time import time
from sklearn.utils import class_weight

# tensorflow and keras
import tensorflow as tf
from tensorflow import keras
from keras import Model
from keras.layers import Dense
from keras.backend.tensorflow_backend import set_session
from keras import metrics
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint

# models
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.inception_resnet_v2 import InceptionResNetV2

# Data utils
from keras.preprocessing.image import ImageDataGenerator

print(tf.__version__)

1.13.1


Using TensorFlow backend.


In [2]:
# gpu stuff
%env CUDA_VISIBLE_DEVICES=0,1
tf.reset_default_graph()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)

env: CUDA_VISIBLE_DEVICES=0,1


In [3]:
# Seeds
tf.random.set_random_seed(123)
np.random.seed(123)

In [4]:
# Hyperparameters
batch_size = 10
train_epochs = 200
architecture = 'mobilenetv2'
weighted_loss = False

In [5]:
#Data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=20, 
                                   zoom_range=0.15,
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2,
                                  horizontal_flip=True,
                                  vertical_flip = True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        'data/val',
        target_size=(224,224),
        batch_size=1,
        class_mode='categorical')

Found 22794 images belonging to 9 classes.
Found 2537 images belonging to 9 classes.


In [6]:
N_train = train_generator.n
N_val = validation_generator.n
class_names = sorted(list(train_generator.class_indices.keys()))
N_classes = len(class_names)

In [7]:
N_train,N_val,class_names

(22794, 2537, ['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'UNK', 'VASC'])

In [8]:
# Weighted loss - not useful here
if(weighted_loss):
    Y_train = train_generator.classes
    class_weight = class_weight.compute_class_weight('balanced',np.unique(Y_train),Y_train)
    class_weight = dict(enumerate(class_weight))
    print(class_weight)
else:
    class_weight = None

In [ ]:
if(architecture=='mobilenetv2'):
    net = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224,224,3), pooling='avg', classes=N_classes)
elif(architecture=='inception_resnet'):
    net = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(299,299,3), pooling='avg', classes=N_classes)
    
x = net.layers[-1].output
fc = Dense(N_classes,activation='softmax')(x)
model = Model(inputs=net.input, outputs=fc)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
show_summary = False
if(show_summary):
    model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
name = architecture+('_weighted')*weighted_loss
checkpointer = ModelCheckpoint(filepath=os.path.join("models",name+"_weights.hdf5"), 
                               monitor = 'val_acc',
                               verbose=1, 
                               save_best_only=True)

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=N_train//train_generator.batch_size,
        epochs=train_epochs,
        validation_data=validation_generator,
        validation_steps=N_val//validation_generator.batch_size,
        class_weight = class_weight,
        verbose= 1,
        shuffle = True,
        callbacks= [tensorboard,checkpointer])

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
 294/2279 [==>...........................] - ETA: 12:52 - loss: 1.2960 - acc: 0.5476

In [ ]:
with open('models/'+name+'_architecture.json', 'w') as f:
    f.write(model.to_json())